In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  9 15:41:43 2020.

@author: tagir.farkhutdinov@atco.com
"""

!pip install sympy==1.5.1

import sympy
from IPython.display import display, Latex

print(__doc__)



Created on Thu Apr  9 15:41:43 2020.

@author: tagir.farkhutdinov@atco.com



In [80]:
"""Print equations."""
# pylint: disable=C0103, R0914
_sp = sy = sympy
sympify = sympy.sympify
x, t, rho_f, rho_s0, g_0, K = _sp.symbols('x t rho_f rho_s^0 g_0 K')
X, Y, P = _sp.symbols('X Y P', cls=_sp.Function)

X = X(x, t)
Y = Y(x, t)

u_s, u_f = _sp.symbols('u_s u_f', cls=_sp.Function)
u_s, u_f = u_s(x, t), u_f(x, t)
g, rho_s = _sp.symbols('g rho_s')

def lhs(u, denom):
    return ((u.diff(t) + u*u.diff(x)) * denom.diff(x)
            ).simplify().factor(fraction=False)

def simplify_eqs(eq):
    return _sp.Eq((-eq.lhs).expand().factor(fraction=False),
                  -eq.rhs)

p_dx_expr = P(X.diff(x), Y.diff(x)).diff(x)
p_dx = _sp.Symbol("\\partial_{x} P")
sigma_s_dx = _sp.Symbol("\\partial_{x}(\\sigma_e + S)")

def eq_builder(u_f, u_s, g, rho_s, rho_f):
    eq_fluid = _sp.Eq(rho_f*g*lhs(u_f, x),
                      (-g*sympify('(p(x,t) + mu(x, t))').diff(x) + K*(u_s - u_f)))
    eq_solid = _sp.Eq(rho_s*lhs(u_s, x),
                      (+g*sympify('p(x,t)').diff(x) - (1-g)*sympify('mu(x,t)').diff(x) + sigma_s_dx + K*(u_f - u_s)))
    eq_p_dx = _sp.Eq(-p_dx, -p_dx_expr)
    return eq_fluid, eq_solid, eq_p_dx

def subscr_partials(eq, before, after):
    return eq.subs(sympify(before).diff(t, t), sympify(after + '_tt')
                  ).subs(sympify(before).diff(x, x), sympify(after + '_xx')
                        ).subs(sympify(before).diff(x, t), sympify(after + '_xt')
                              ).subs(sympify(before).diff(x), sympify(after + '_x')
                                    ).subs(sympify(before).diff(t), sympify(after + '_t')
                                          )
                                     
def nice_partials(eq):
    return subscr_partials(subscr_partials(eq, 'Y(x, t)', 'Y'), 'X(x, t)', 'X')

def print_eqn(eq, oneline=True, nicepartials=True):
    # pylint: disable=W0212
    eq = nice_partials(eq)
    def correct_render(latex_repr):
        return latex_repr.replace(
            '\\substack', '').replace(
                '\\displaystyle', '').replace(
                    r'\xi_{1}', r'\xi').replace('\\xi_{2}', '\\eta')
    if not oneline:
        display(Latex(correct_render(eq.lhs._repr_latex_())))
        display(Latex('$\\quad=' +
                      correct_render(eq.rhs._repr_latex_()[1:-1])
                      + ',$'))
    else:
        display(Latex(correct_render(eq._repr_latex_())))


In [81]:
# Original equations
eq_fluid, eq_solid, eq_p_dx = eq_builder(u_f, u_s, g, rho_s, rho_f)
print('Fluid dynamics:')
print_eqn(eq_fluid, True)
print('Solid dynamics:')
print_eqn(eq_solid, True)
# Substitutions of X and Y
u_s_ = -X.diff(t)/X.diff(x)
u_f_ = -Y.diff(t)/Y.diff(x)
g_ = g_0 * Y.diff(x)
rho_s_ = rho_s0 * X.diff(x)
eq_fluid, eq_solid, eq_p_dx = eq_builder(u_f_, u_s_, g_, rho_s_, rho_f)

eq_fluid, eq_solid, eq_p_dx = list(
    map(simplify_eqs, [eq_fluid, eq_solid, eq_p_dx]))

print('Dynamics of fluid and solid after substitution:')
print_eqn(eq_fluid, True)
print_eqn(eq_solid, True)
print('Pressure:')
print_eqn(eq_p_dx, True)

Fluid dynamics:


<IPython.core.display.Latex object>

Solid dynamics:


<IPython.core.display.Latex object>

Dynamics of fluid and solid after substitution:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

Pressure:


<IPython.core.display.Latex object>

In [117]:
print("Total incompressibility:")
print("DON'T WORRY about missing braces after partials x,\nit's solely a sympy representation bug):")

def eq_incompr(g, u_f, u_s): return sy.Eq(sy.Derivative(g*u_f, x),
                                          -sy.Derivative(((1-g)*u_s), x))
print_eqn(eq_incompr(g, u_f, u_s), True)

print("After substitutions:")
print_eqn(eq_incompr(g_, u_f_, u_s_), True)
print_eqn(eq_incompr(g_, u_f_, u_s_).doit(), True)
print("Multipled by X_x**2:")
print_eqn(sy.Eq((X.diff(x)**2 * 
                (eq_incompr(g_, u_f_, u_s_).doit().lhs + g_0*X.diff(t)*Y.diff(x, x)/X.diff(x))),
                (X.diff(x)**2 * 
                 (eq_incompr(g_, u_f_, u_s_).doit().rhs + g_0*X.diff(t)*Y.diff(x, x)/X.diff(x))
                )
               ).simplify()
         )

print("In our paper")
print_eqn(sy.Eq(g_0 * X.diff(x) * Y.diff(t) + (1 - g_0 * Y.diff(x)) * X.diff(t), 0))
print("- Partial wrt x")
print_eqn(sy.Eq(-(g_0 * X.diff(x) * Y.diff(t) + (1 - g_0 * Y.diff(x)) * X.diff(t)).diff(x), 0))

print("It seems that it should be ")
eq_incompr_correct = sy.Eq(
    -g_0 * sy.Derivative(Y.diff(t)/X.diff(x), x) * X.diff(x)**3,
    (1 - g_)*sy.Derivative(X.diff(t)/X.diff(x), x) * X.diff(x)**2
)
print_eqn(eq_incompr_correct)
print("Partial wrt x")
print_eqn(sy.Eq((eq_incompr_correct.lhs.doit()).simplify(),
                (eq_incompr_correct.rhs.doit()).simplify())
         )

Total incompressibility:
DON'T WORRY about missing braces after partials x,
it's solely a sympy representation bug):


<IPython.core.display.Latex object>

After substitutions:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

It seems that it should be 


<IPython.core.display.Latex object>

Partial wrt x


<IPython.core.display.Latex object>

In [ ]:
# Total incompressibility
incompr = 

In [18]:
print('Potential energy:')
rho_s, rho_f = X.diff(x), g/g_0

def potential_rhs(rho_s, rho_f):
    return (
        _sp.Symbol('alpha')/2 * (rho_s - 1)**2
        + _sp.Symbol('beta')/2 * (rho_f*g_0 - (1-(1-g_0)*rho_s))**2
    )

eq_potential = _sp.Eq(_sp.symbols('V', cls=sy.Function)(rho_s, rho_f),
                      potential_rhs(rho_s, rho_f)
                      )
print_eqn(eq_potential, True)

Potential energy:


<IPython.core.display.Latex object>

In [19]:
b = _sp.Symbol('b')
rho_s = 1/_sp.sqrt(b)
eq_potential = _sp.Eq(_sp.symbols('V', cls=sy.Function)(rho_s, rho_f),
                      potential_rhs(rho_s, rho_f)
                      )
display(eq_potential)

v = _sp.Symbol('v')
eq_potential = _sp.Eq(_sp.symbols('V', cls=sy.Function)(b, v),
                      potential_rhs(rho_s,
                                    _sp.Symbol('c_0')*v/_sp.sqrt(b))
                      )
display(eq_potential)

Eq(V(1/sqrt(b), Derivative(Y(x, t), x)), alpha*(-1 + 1/sqrt(b))**2/2 + beta*(g_0*Derivative(Y(x, t), x) - 1 + (1 - g_0)/sqrt(b))**2/2)

Eq(V(b, v), alpha*(-1 + 1/sqrt(b))**2/2 + beta*(-1 + c_0*g_0*v/sqrt(b) + (1 - g_0)/sqrt(b))**2/2)

In [20]:
print('Sigma epsilon:')
V = _sp.symbols('V', cls=_sp.Function)
sigma_eps = _sp.Eq(V(b, v) + 2*V(b, v).diff(b)*b,
                   (2*(eq_potential.rhs.diff(b))*b +
                   eq_potential.rhs)
                   )
print_eqn(sigma_eps)
sigma_eps = _sp.Eq(sigma_eps.lhs, sigma_eps.rhs.expand())
print_eqn(sigma_eps)

print('Partial_x sigma epsilon:')
partial_sigma_eps = _sp.Eq(_sp.Symbol('\\partial_{x} \\sigma_e'),
                           _sp.Symbol('\\partial_{x}')
                           * sigma_eps.rhs.subs(
                               b, 1/X.diff(x)**2
                               ).subs(
                                   v,
                                   _sp.Symbol('v_0')*Y.diff(x)/X.diff(x)))
print_eqn(partial_sigma_eps)
print('or, the whole expression = ')
dsigmadx = partial_sigma_eps.rhs.diff(x)/_sp.Symbol('\\partial_{x}')
# print(type(dsigmadx.args[0]))
display(dsigmadx.args[0])
print('+')
display((dsigmadx - dsigmadx.args[0]
         ).subs(_sp.Symbol('v_0'), g_0
                ).subs(_sp.Symbol('c_0'), 1).factor())

print('We used the following formula for pore volume in derivation above:')
pore_vol_eq = _sp.Eq(_sp.Symbol('v')/_sp.Symbol('v_0'),
                     Y.diff(x)/X.diff(x))
display(pore_vol_eq)

print("We'll use the same formula to find pressure")
pressure_eq = _sp.Eq(_sp.Symbol('pc(b)'),
                     eq_potential.rhs.diff(v))
display(pressure_eq)

print("or, for pressure we have")
pressure_eq = _sp.Eq(_sp.Symbol('p'),
                     pressure_eq.rhs/_sp.Symbol('c_0')*_sp.sqrt(b))

display(pressure_eq)
pressure_eq = _sp.Eq(_sp.Symbol('p'),
                     pressure_eq.rhs.subs(
                         _sp.sqrt(b), 1/X.diff(x)
                         ).subs(_sp.Symbol('v'),
                                _sp.Symbol('v_0') * Y.diff(x)/X.diff(x)))
display(pressure_eq)

print("and subsequently")
pressure_eq = _sp.Eq(_sp.Symbol('\\partial_{x} p'),
                     pressure_eq.rhs.diff(x))
display(pressure_eq)




Sigma epsilon:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

or, for pressure we have


Eq(p, beta*g_0*(-1 + c_0*g_0*v/sqrt(b) + (1 - g_0)/sqrt(b)))

Eq(p, beta*g_0*(c_0*g_0*v_0*Derivative(Y(x, t), x) + (1 - g_0)*Derivative(X(x, t), x) - 1))

and subsequently


Eq(\partial_{x} p, beta*g_0*(c_0*g_0*v_0*Derivative(Y(x, t), (x, 2)) + (1 - g_0)*Derivative(X(x, t), (x, 2))))